In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/output.xlsx")
df = df.drop(columns='Unnamed: 0') #ненужный столбец с нумерацией записей, избавляемся от него

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df.head

<bound method NDFrame.head of                                                   Name  ...     Subgenre
0                                    Cutting for Stone  ...        Novel
1                                    Cutting for Stone  ...        Novel
2    Milk and Vine: Inspirational Quotes From Class...  ...  Non Fiction
3                                 Go the F**k to Sleep  ...       Comedy
4    What Should Danny Do? (The Power to Choose Ser...  ...     Children
..                                                 ...  ...          ...
545             Player's Handbook (Dungeons & Dragons)  ...        Games
546             Player's Handbook (Dungeons & Dragons)  ...        Games
547             Player's Handbook (Dungeons & Dragons)  ...        Games
548  Divine Soul Mind Body Healing and Transmission...  ...  Non Fiction
549  Soul Healing Miracles: Ancient and New Sacred ...  ...  Non Fiction

[550 rows x 8 columns]>

In [3]:
df.dtypes

Name            object
Author          object
User Rating    float64
Reviews          int64
Price            int64
Year             int64
Genre           object
Subgenre        object
dtype: object

В изначальном датасете было разделение только на жанры художественной литературы и не художественной. Чтобы было интереснее анализировать книги попавшие в топ я решил вручную разделить художественную литературу по жанрам в столбце Subgenre (это было быстрее сделать вручную чем парсингом, т.к. не все книги есть на википедии и не у всех отмечен жанр). У не художественных книг в этом столбце остается категория Non Fiction Пример записи из обновленного датасета:

In [4]:
df[:1]

,Name,Author,User Rating,Reviews,Price,Year,Genre,Subgenre
0,Cutting for Stone,Abraham Verghese,4.6,4866,11,2011,Fiction,Novel


Посмотрим, сколько всего поджанров есть в датасете

In [5]:
subg = set(df['Subgenre'].to_list())
print(subg)
print(len(subg))

{'Comics', 'Non Fiction', 'Young Adult fiction', 'Horror', 'Young Adult Fiction', 'Drama', 'Thriller', 'Science fiction', 'Comedy', 'Games', 'Children', 'Historical', 'Fantasy', 'Novel'}
14


Видим что Young Adult fiction и Young Adult Fiction отличается только регистром, приведем записи с этим полем к одному виду чтобы избежать повторений

In [6]:
for i in range(0,549):
  if df['Subgenre'][i]=='Young Adult fiction':
    df['Subgenre'][i]='Young Adult Fiction'
subg = set(df['Subgenre'].to_list())
print(subg)

{'Comics', 'Non Fiction', 'Horror', 'Young Adult Fiction', 'Drama', 'Thriller', 'Science fiction', 'Comedy', 'Games', 'Children', 'Historical', 'Fantasy', 'Novel'}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
df.dtypes

Name            object
Author          object
User Rating    float64
Reviews          int64
Price            int64
Year             int64
Genre           object
Subgenre        object
dtype: object

Проанализируем, какие данные мы ожмдаем увидеть в датасете:

**Name** - название книги, строка, в датафрейме данное поле имеет тип object что является верным, интервала определеных значений не имеет

**Author** - имя автора, строка, в датафрейме данное поле имеет тип object что является верным, интервала определеных значений не имеет

**User rating** - средний рейтинг пользователей, число с плавающей точкой, в датафрейме данное поле имеет тип float64 что является верным, интервал значений (0.0,5.0]

**Reviews** - число отзывов на книгу, целое число, в датафрейме данное поле имеет тип int64 что является верным, заданного интервала значений не имеет

**Price** - цена книги, целое число, в датафрейме данное поле имеет тип int64 что является верным, интервал значений >0

**Year** - год в котором книга попала в топ, целое число, в датафрейме данное поле имеет тип int64 что является верным, диапазон значений {2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019}

**Genre** - жанр книги, строка, в датафрейме данное поле имеет тип object что является верным, может принимать только значения Fiction/Non Fiction

Subgenre - поджанр книги, строка, в датасете имеет тип object что является верным, диапазон значений {'Thriller', 'Comedy', 'Non Fiction', 'Comics', 'Games', 'Young Adult Fiction', 'Historical', 'Children', 'Science fiction', 'Novel', 'Fantasy', 'Horror', 'Drama'}


Проверим данные в столбце 'Genre' т.к. там должно быть только два вида записей

In [8]:
df['Genre'].unique()

array(['Fiction', 'Non Fiction'], dtype=object)

Данные в столбцах Name, Author, Price, Reviews не находятся в пределах каких-то интервала, а вот столбцы User Rating и Year нужно проверить

In [9]:
df['User Rating'].unique()

array([4.6, 4.4, 4.8, 4.7, 4.5, 4.9, 4.3, 4.2, 4.1, 3.9, 3.8, 4. , 3.6,
       3.3])

Выхода за интервал значений нет, но проверяем, что 4. это то же самое что 4.0 и 4, а не ошибка данных

In [10]:
df.loc[df['User Rating'] == 4 ]

,Name,Author,User Rating,Reviews,Price,Year,Genre,Subgenre
180,StrengthsFinder 2.0,Gallup,4.0,5069,17,2013,Non Fiction,Non Fiction
181,StrengthsFinder 2.0,Gallup,4.0,5069,17,2010,Non Fiction,Non Fiction
182,StrengthsFinder 2.0,Gallup,4.0,5069,17,2009,Non Fiction,Non Fiction
183,StrengthsFinder 2.0,Gallup,4.0,5069,17,2014,Non Fiction,Non Fiction
184,StrengthsFinder 2.0,Gallup,4.0,5069,17,2015,Non Fiction,Non Fiction
185,StrengthsFinder 2.0,Gallup,4.0,5069,17,2016,Non Fiction,Non Fiction
186,StrengthsFinder 2.0,Gallup,4.0,5069,17,2017,Non Fiction,Non Fiction
187,StrengthsFinder 2.0,Gallup,4.0,5069,17,2011,Non Fiction,Non Fiction
188,StrengthsFinder 2.0,Gallup,4.0,5069,17,2012,Non Fiction,Non Fiction
216,Gone Girl,Gillian Flynn,4.0,57271,9,2014,Fiction,Thriller


Проверим, что в столбце Price нет нулевых значений:

In [11]:
df.loc[df['Price'] <= 0 ]

,Name,Author,User Rating,Reviews,Price,Year,Genre,Subgenre
9,Little Blue Truck,Alice Schertle,4.9,1884,0,2014,Fiction,Children
116,The Constitution of the United States,Delegates of the Constitutional,4.8,2774,0,2016,Non Fiction,Non Fiction
225,Go Set a Watchman: A Novel,Harper Lee,4.8,26234,0,2014,Fiction,Novel
227,To Kill a Mockingbird,Harper Lee,4.8,26234,0,2015,Fiction,Novel
228,To Kill a Mockingbird,Harper Lee,4.8,26234,0,2016,Fiction,Novel
229,To Kill a Mockingbird,Harper Lee,4.8,26234,0,2013,Fiction,Novel
258,Diary of a Wimpy Kid: The Long Haul,Jeff Kinney,4.8,4505,0,2011,Fiction,Children
260,Double Down (Diary of a Wimpy Kid #11),Jeff Kinney,4.8,5836,0,2017,Fiction,Children
264,"The Third Wheel (Diary of a Wimpy Kid, Book 7)",Jeff Kinney,4.8,6812,0,2013,Fiction,Children
404,Frozen (Little Golden Book),RH Disney,4.7,3642,0,2014,Fiction,Children


К сожалению, такие нулевые значения есть. Однако после проверки данных на сайте Amazon я выяснил, что данная цена реальна и в датасете указывается самая дешевая стоимость(а т.к. самая дешевая стоимость в приложении Amazon с электронными книгами, то некоторые книги действительно бесплатные.

Проверям столбец Year

In [12]:
df['Year'].unique()

array([2011, 2010, 2017, 2019, 2016, 2009, 2014, 2013, 2015, 2018, 2012])

В стобце Year никаких отклонений нет. Проверим весь датафрейм на наличие пустых значений:

In [13]:
df.isnull().values.any()

False

Пустых значений нет, можем сделать вывод, что все данные в датасете имеют корректный тип и находятся в пределах допустимых значений